In [1]:
import fitz
import re
import mariadb
import sys
import pandas as pd
import numpy as np
import mwparserfromhell
import xml.etree.ElementTree as ET

In [ ]:
wiki_corpus = list()
# Read the file
with open('wikiex\\wiki_00', 'r', encoding='utf-8') as file: # for wiki_00, wiki_01, wiki_02
    lines = file.readlines()

# Filter lines that do not start with a curly bracket
filtered_lines = [line for line in lines if not line.strip().startswith('<') and line.strip()]
wiki_corpus.extend(filtered_lines)
wiki_corpus = [line.replace('\n','') for line in wiki_corpus]
with open('wikiex\\wiki_corpus.txt', 'w', encoding='utf-8') as f:
    for item in wiki_corpus:
        f.write("%s\n" % item)

# Prompts

#I need to parse olo wiki database in xml format. It's a long file in xml format consists of: Articles, templates, media/file descriptions, and primary meta-pages. Get me only articles and their titles

In [ ]:
# Parse the XML file
wiki_path = 'olowiki-20250201-pages-articles-multistream.xml'
tree = ET.parse(wiki_path)
root = tree.getroot()

In [20]:
root.findall('text')

[]

In [4]:
# Extract articles and their titles
articles = []
for page in root.findall('.//page'):
    ns = page.find('ns').text
    if ns == '0':  # Namespace 0 indicates an article
        title = page.find('title').text
        text = page.find('.//text').text
        articles.append({'title': title, 'text': text})

# Print the articles and their titles
for article in articles:
    print(f"Title: {article['title']}")
    print(f"Text: {article['text'][:200]}...")  # Print first 200 characters of the article text
    print()

In [41]:
def strip_tag_name(t):
    idx = k = t.rfind("}")
    return t[idx + 1:] if idx != -1 else t

def clean_wiki_text(text):
    """Очищает вики-разметку и HTML-теги из текста"""
    if not text:
        return ""
    # Удаляем шаблоны {{...}} (например, {{Infobox}})
    text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    # Удаляем вики-ссылки [[...|...]] и [[...]]
    text = re.sub(r"\[\[([^|\]]*\|)?([^\]]+)\]\]", r"\2", text)
    # Убираем жирный и курсивный текст '''...''' и ''...''
    text = re.sub(r"''+", "", text)
    # Убираем заголовки == ... ==, === ... ===
    text = re.sub(r"={2,}.*?={2,}", "", text)
    # Удаляем HTML-теги
    text = re.sub(r"<.*?>", "", text)
    # Убираем лишние пробелы и пустые строки
    text = re.sub(r"\n{2,}", "\n", text).strip()
    return text

events = ("start", "end")

title = None
texts, texts_clean, titles = list(), list(), list()
for event, elem in ET.iterparse(wiki_path, events=events):
    tname = strip_tag_name(elem.tag)

    if event == 'end':
        if tname == 'title':
            title = elem.text
            if title is not None:
                print(f'Заголовок: {title}')
                titles.append(title)
        elif tname == 'text':
            text = elem.text
            if text is not None:
                # wikicode = mwparserfromhell.parse(text)  # Парсим вики-разметку
                # clean_text = wikicode.strip_code()
                clean_text = clean_wiki_text(text)
                print(clean_text)
                texts_clean.append(clean_text)
                texts.append(text)

    elem.clear()

__NEWSECTIONLINK__ __FORCETOC__
* olo: .
* en: Requests for the bot flag should be made on this page. This wiki uses the standard bot policy, and allows global bots and automatic approval of certain types of bots. Other bots should apply below, and then request access from a steward if there is no objection.
С 12 июня по 12 июля будет проходить вики-марафон «Вики любит футбол 2020». Именно в эти сроки должны были пройти два крупнейших континентальных футбольных турнира — Евро-2020 и Кубок Америки 2020. Однако из-за коронавируса оба турнира были перенесены на 2021 год. Объявляя этот конкурс, википедисты намерены выразить свою солидарность с футбольным сообществом, а также восполнить пробелы, связанные с футбольной тематикой — особенно в разделах на языках народов России. Марафон организован НП «Викимедиа РУ», Группой «Викимедийцы Башкортостана», Группой участников-википедистов Северного Кавказа, Группой «Викимедийцы, пишущие на эрзянском языке», Группой татароязычных участников сообщест

In [49]:
# Загружаем XML-дамп
tree = ET.parse(wiki_path)  # Укажите путь к вашему файлу
root = tree.getroot()

# Обрабатываем каждую страницу
for page in root.findall('.//page'):
    title = page.find('title')  # Заголовок статьи
    print(title)
    text = page.find('.//text').text  # Текст статьи с разметкой
    if text:  # Проверяем, что текст есть
        wikicode = mwparserfromhell.parse(text)  # Парсим вики-разметку
        clean_text = wikicode.strip_code()  # Удаляем разметку
        # Сохраняем текст в файл
        with open(f"{title}.txt", "w", encoding="utf-8") as f:
            f.write(clean_text)

In [ ]:
def parse_olo_wiki(xml_file):
    """Parses an Olo Wikipedia XML dump and extracts articles with their titles."""
    articles = []
    
    context = ET.iterparse(xml_file, events=("start", "end"))
    _, root = next(context)  # Get root element
    
    for event, elem in context:
        if event == "end" and elem.tag == "page":
            title = elem.find("title").text
            print(title)
            namespace = elem.find("ns").text  # Namespace identifier
            text_elem = elem.find("revision/text")
            
            if namespace == "0" and text_elem is not None:  # Only main namespace (articles)
                text = text_elem.text if text_elem.text else ""
                articles.append((title, text))
            
            elem.clear()  # Free up memory
    
    return articles
articles = parse_olo_wiki(wiki_path)
    
for title, content in articles[:10]:  # Print first 10 articles as a sample
    print(f"Title: {title}\nContent: {content[:200]}...\n")

In [ ]:
# Define the target title
target_title = "Revinnyh kottu"

# Parse the XML file
tree = ET.parse(wiki_path)
root = tree.getroot()

# Find and extract the target articles
target_articles = []
for page in root.findall('.//page'):
    print(page)

In [62]:
from bs4 import BeautifulSoup

# Load and parse the XML file
with open(wiki_path, 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'xml')

# Extract articles and their titles
articles = []
for page in soup.find_all('page'):
    ns = page.find('ns').text
    if ns == '0':  # Namespace 0 indicates an article
        title = page.find('title').text
        text = page.find('text').text
        wikicode = mwparserfromhell.parse(text)  # Парсим вики-разметку
        clean_text = wikicode.strip_code()
        articles.append({'title': title, 'text': text, 'clean_text':clean_text})

# Print the articles and their titles
for article in articles:
    print(f"Title: {article['title']}")
    print(f"Text: {article['text'][:200]}...")  # Print first 200 characters of the article text
    print()

Title: Piäsivu
Text: __NOTOC__
__NOEDITSECTION__
{|id="EnWpMpBook2" class="MainPageLetterHead" style="width:100%; background-repeat: no-repeat; background-position: 0% 20%;background-color: #fafafd; border: 1px solid #ccc...

Title: 1. elokuudu
Text: {{MonthCalendar
|date=08/01/{{CURRENTYEAR}}
|float=right
|color=#ccccff
|color2=#ccccff
|EndNote=<small>[[{{CURRENTYEAR}}|Vuozi {{CURRENTYEAR}}]]</small>
}}
1. elokuudu — 213. vuvven päivy (214. päivy...

Title: 1. heinykuudu
Text: {{MonthCalendar
|date=07/01/{{CURRENTYEAR}}
|float=right
|color=#ccccff
|color2=#ccccff
|EndNote=<small>[[{{CURRENTYEAR}}|Vuozi {{CURRENTYEAR}}]]</small>
}}
'''1. heinykuudu''' - 182. vuvven päivy (18...

Title: 1. kevätkuudu
Text: {{MonthCalendar
|date=03/01/{{CURRENTYEAR}}
|float=right
|color=#ccccff
|color2=#ccccff
|EndNote=<small>[[{{CURRENTYEAR}}|Vuozi {{CURRENTYEAR}}]]</small>
}}
'''1. kevätkuudu''' - 60. vuvven päivy (61....

Title: 1. kezäkuudu
Text: {{MonthCalendar
|date=06/01/{{CURRENTYEAR}}
|float=rig